In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("quora.csv")
df.dropna(inplace=True)

df = df.drop(['id', 'qid1', 'qid2'], axis=1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

q_df = df.drop(['is_duplicate'], axis=1)

questions = list(q_df['question1']) + list(q_df['question2'])

cv = CountVectorizer(max_features=3000)

transformed_ques = cv.fit_transform(questions).toarray()

q1_transf = transformed_ques[:len(q_df['question1'])]
q2_transf = transformed_ques[len(q_df['question2']):]

q1_transf_df = pd.DataFrame(q1_transf, index= q_df.index)
q2_transf_df = pd.DataFrame(q2_transf, index= q_df.index)

q_transf_df = pd.concat([q1_transf_df, q2_transf_df], axis=1)

q_transf_df.shape